In [1]:
import os

# IMPORTANT: Replace with your actual Twitch application credentials
os.environ["TWITCH_CLIENT_ID"] = "pgapguj2m5e5mm637jg1sx952kjdzw"
os.environ["TWITCH_CLIENT_SECRET"] = "du7gazte4165jh8rn8w4pao37zpazi"

print("TWITCH_CLIENT_ID and TWITCH_CLIENT_SECRET environment variables set.")

TWITCH_CLIENT_ID and TWITCH_CLIENT_SECRET environment variables set.


In [2]:
import pandas as pd
import os
from pathlib import Path

# Find the latest games CSV file in the 'igdb_csv' directory
outdir = Path("igdb_csv")
if outdir.exists():
	try:
		latest_games_file = sorted(outdir.glob("games_*.csv"), reverse=True)[0]
		print(f"Loading data from: {latest_games_file}")
		df = pd.read_csv(latest_games_file)
		print(df.head())  # show first 5 rows
	except IndexError:
		print("No games CSV file found in 'igdb_csv' directory.")
		print("Please run the data download cell first.")
else:
	print("'igdb_csv' directory not found.")
	print("Please run the data download cell first to create it.")

Loading data from: igdb_csv/games_20251015_1657.csv
   id  aggregated_rating aggregated_rating_count first_release_date   genres  \
0   1          90.000000                     1.0          953596800  5|13|31   
1   2          70.000000                     1.0          912384000    13|31   
2   3          83.000000                     2.0         1085443200  5|13|31   
3   4          63.642857                    14.0         1393286400     5|31   
4   5                NaN                     NaN          914198400       12   

                      name        platforms                    slug  \
0  Thief II: The Metal Age                6  thief-ii-the-metal-age   
1  Thief: The Dark Project                6  thief-the-dark-project   
2    Thief: Deadly Shadows             11|6    thief-deadly-shadows   
3                    Thief  9|48|6|14|12|49                   thief   
4            Baldur's Gate        3|6|39|14           baldur-s-gate   

   total_rating  total_rating_count    u

In [5]:
import pandas as pd
from pathlib import Path
import numpy as np

# Adjust timestamp to your generated CSVs
ts = "20251015_1600"  
outdir = Path("igdb_csv")

games = pd.read_csv(outdir / f"games_20251001_1633.csv")
popularity_primitives = pd.read_csv(outdir / f"popularity_primitives_20251015_1657.csv")
popularity_types = pd.read_csv(outdir / f"popularity_types_20251001_1809.csv")

# Adding game_time_to_beats to explore top games based on playtime weighted popularity score
game_time_to_beats = pd.read_csv(outdir / f"game_time_to_beats_20251015_1657.csv")

In [6]:
# Exploratory Idea #2: Analyze top 10 video games based on playtime weighted popularity score
# What are the most popular video games based on player engagement; Looking into how long people play specific video games for

In [10]:
# STEP 1: Load merged popularity and game info from 'Game_Popularity' notebook
# Unnecessary to merge and clean columns again from same csv file merge

popularity_with_games = pd.read_csv("igdb_popularity_by_game.csv")

In [13]:
# STEP 2: Merge 'popularity_with_games' with playtime data
# Merge on game_id (linking time-to-beat and popularity)
playtime_popularity = popularity_with_games.merge(
    game_time_to_beats[["game_id", "normally", "hastily", "completely"]],
    on="game_id",
    how="left"
)

In [ ]:

# --- 4️⃣ Compute average playtime ---
playtime_popularity["avg_playtime"] = playtime_popularity[
    ["normally", "hastily", "completely"]
].mean(axis=1, skipna=True)

# --- 5️⃣ Compute playtime-based popularity score ---
# Combine popularity value with engagement
playtime_popularity["playtime_popularity"] = (
    playtime_popularity["value"] * np.log1p(playtime_popularity["avg_playtime"])
)

# --- 6️⃣ Aggregate by game ---
game_popularity_summary = (
    playtime_popularity
    .groupby("game_name", as_index=False)
    .agg({
        "value": "mean",
        "avg_playtime": "mean",
        "playtime_popularity": "mean"
    })
    .sort_values("playtime_popularity", ascending=False)
    .head(10)
)